In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e10/sample_submission.csv
/kaggle/input/playground-series-s4e10/train.csv
/kaggle/input/playground-series-s4e10/test.csv


# Dataset

In [2]:
df = pd.read_csv("/kaggle/input/playground-series-s4e10/train.csv")

df[:10]

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0
5,5,27,45000,RENT,2.0,VENTURE,A,9000,8.94,0.20,N,5,0
6,6,25,45000,MORTGAGE,9.0,EDUCATION,A,12000,6.54,0.27,N,3,0
7,7,21,20000,RENT,0.0,PERSONAL,C,2500,13.49,0.13,Y,3,0
8,8,37,69600,RENT,11.0,EDUCATION,D,5000,14.84,0.07,Y,11,0
9,9,35,110000,MORTGAGE,0.0,DEBTCONSOLIDATION,C,15000,12.98,0.14,Y,6,0


## Handel missing values

In [3]:
numeric_cols = [
    'person_age', 'person_income', 'person_emp_length', 'loan_amnt',
    'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length'
]

# Option 1: Assign directly to the column
for col in numeric_cols:
    median_val = df[col].median()
    df[col] = df[col].fillna(median_val)

In [4]:


categorical_cols = [
    'person_home_ownership', 'loan_intent', 'loan_grade', 
    'cb_person_default_on_file'
]

# Option 1: Assign directly to the column
for col in categorical_cols:
    mode_val = df[col].mode()[0]
    df[col] = df[col].fillna(mode_val)

In [5]:

# Verify that all missing values are handled
print("Missing values after imputation:")
print(df.isnull().sum())

Missing values after imputation:
id                            0
person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_amnt                     0
loan_int_rate                 0
loan_percent_income           0
cb_person_default_on_file     0
cb_person_cred_hist_length    0
loan_status                   0
dtype: int64


## Encode categorical_cols

### The get_dummies() function handles the conversion automatically for us, convert it to one hode code

In [6]:
categorical_cols_to_encode = [
    'person_home_ownership', 
    'loan_intent', 
    'loan_grade', 
    'cb_person_default_on_file', 
    
]

In [7]:
df_encoded = pd.get_dummies(df, columns=categorical_cols_to_encode, drop_first=True)


print(df_encoded.head())

   id  person_age  person_income  person_emp_length  loan_amnt  loan_int_rate  \
0   0          37          35000                0.0       6000          11.49   
1   1          22          56000                6.0       4000          13.35   
2   2          29          28800                8.0       6000           8.90   
3   3          30          70000               14.0      12000          11.11   
4   4          22          60000                2.0       6000           6.92   

   loan_percent_income  cb_person_cred_hist_length  loan_status  \
0                 0.17                          14            0   
1                 0.07                           2            0   
2                 0.21                          10            0   
3                 0.17                           5            0   
4                 0.10                           3            0   

   person_home_ownership_OTHER  ...  loan_intent_MEDICAL  \
0                        False  ...               

In [8]:
print("\nDataFrame columns after one-hot encoding:")
print(df_encoded.columns)


DataFrame columns after one-hot encoding:
Index(['id', 'person_age', 'person_income', 'person_emp_length', 'loan_amnt',
       'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length',
       'loan_status', 'person_home_ownership_OTHER',
       'person_home_ownership_OWN', 'person_home_ownership_RENT',
       'loan_intent_EDUCATION', 'loan_intent_HOMEIMPROVEMENT',
       'loan_intent_MEDICAL', 'loan_intent_PERSONAL', 'loan_intent_VENTURE',
       'loan_grade_B', 'loan_grade_C', 'loan_grade_D', 'loan_grade_E',
       'loan_grade_F', 'loan_grade_G', 'cb_person_default_on_file_Y'],
      dtype='object')


##  Check for imbalance

###  How to Check for Imbalance

In [9]:
# Assuming your preprocessed DataFrame is named 'df_encoded'

# Count the number of each class in the 'loan_status' column
class_counts = df_encoded['loan_status'].value_counts()

# Print the counts to see the absolute distribution
print("Class distribution of 'loan_status':")
print(class_counts)

# Print the percentages to see the relative distribution
print("\nPercentage distribution:")
print(df_encoded['loan_status'].value_counts(normalize=True) * 100)

Class distribution of 'loan_status':
loan_status
0    50295
1     8350
Name: count, dtype: int64

Percentage distribution:
loan_status
0    85.761787
1    14.238213
Name: proportion, dtype: float64


# Train the model

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X = df_encoded.drop(columns=['id', 'loan_status']) # Make sure to drop the ID and target
y = df_encoded['loan_status']

# Split the data into training and testing sets (e.g., 80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest model
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)



RandomForestClassifier(n_estimators=200, random_state=42)

## Evaluate with Confusion Matrix, Precision, Recall, F1

In [11]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(X_test)

# Generate and print the Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Generate and print the Classification Report (includes Precision, Recall, F1-score)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Confusion Matrix:
[[9979  108]
 [ 454 1188]]

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.97     10087
           1       0.92      0.72      0.81      1642

    accuracy                           0.95     11729
   macro avg       0.94      0.86      0.89     11729
weighted avg       0.95      0.95      0.95     11729



## Result:

### The high precision for class 1 (0.92) : means that when the model predicts a loan will be unapproved, it's very likely to be correct.
###  the lower recall for class 1 (0.72) : means the model is missing a decent number of unapproved loans.


### so, we will try to make hime indecate/identify the unapproved loans better, and that's by # Iterate & improve. phase

# Iterate & improve.

## 1. Address Class Imbalance with a Class Weight


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model with balanced class weights, as it gives more attention to the minorty class
model_weighted = RandomForestClassifier(n_estimators=200, random_state=42, class_weight='balanced')
model_weighted.fit(X_train, y_train)

# Evaluate the new model
from sklearn.metrics import classification_report
y_pred_weighted = model_weighted.predict(X_test)
print("Classification Report with Class Weights:")
print(classification_report(y_test, y_pred_weighted))

Classification Report with Class Weights:
              precision    recall  f1-score   support

           0       0.96      0.99      0.97     10087
           1       0.92      0.71      0.81      1642

    accuracy                           0.95     11729
   macro avg       0.94      0.85      0.89     11729
weighted avg       0.95      0.95      0.95     11729



### THE RECALL AS IT FOR THE CLASS 1   :( :(
### WE WILL TRY THIS METHOD CALLED [SMOTE], TO to address class imbalance, AND SEE WHAT WILL HAPPEN!

## 2. Feature Selection


### we will use feature_importances_ attribute of the Random Forest model to see which features were most influential in making predictions.

In [13]:
# Assuming you have a trained model named 'model' from the previous step

# Get feature importances
feature_importances = model.feature_importances_

# Create a DataFrame for better visualization
feature_importance_df = pd.DataFrame({
    'feature': X_train.columns,
    'importance': feature_importances
}).sort_values(by='importance', ascending=False)

print("Top 10 most important features:")
print(feature_importance_df.head(10))

# Now you can select these top features to train a new model
top_features = feature_importance_df['feature'].head(10).tolist()
X_top = X[top_features]

Top 10 most important features:
                       feature  importance
5          loan_percent_income    0.237661
4                loan_int_rate    0.144234
1                person_income    0.112626
3                    loan_amnt    0.077274
17                loan_grade_D    0.074653
9   person_home_ownership_RENT    0.072416
2            person_emp_length    0.061198
0                   person_age    0.048704
6   cb_person_cred_hist_length    0.038423
16                loan_grade_C    0.017890


## 3. Hyperparameter Tuning


In [14]:
from sklearn.model_selection import GridSearchCV

# Define the hyperparameters to tune
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': ['balanced']  # Using balanced weights as a good starting point
}

# Create a GridSearchCV object
grid_search = GridSearchCV(
    RandomForestClassifier(random_state=42), 
    param_grid, 
    cv=3,  # Number of cross-validation folds
    scoring='recall' # We are optimizing for recall of the minority class
)

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print("Best parameters found: ", grid_search.best_params_)
print("Best recall score: ", grid_search.best_score_)

# Use the best model found by grid search
best_model = grid_search.best_estimator_

# Evaluate the best model on the test set
y_pred_best = best_model.predict(X_test)
print("\nClassification Report with Best Model:")
print(classification_report(y_test, y_pred_best))

Best parameters found:  {'class_weight': 'balanced', 'max_depth': 10, 'min_samples_leaf': 4, 'n_estimators': 100}
Best recall score:  0.7666964818127608

Classification Report with Best Model:
              precision    recall  f1-score   support

           0       0.96      0.95      0.96     10087
           1       0.72      0.79      0.75      1642

    accuracy                           0.93     11729
   macro avg       0.84      0.87      0.85     11729
weighted avg       0.93      0.93      0.93     11729



### OBSERVATION

### Everything is worse now! 

# Bonus

##  SMOTE  to address class imbalance

### What is SMOTE? SMOTE stands for Synthetic Minority Oversampling Technique. The core idea is to generate new data for the minority class, rather than just duplicating existing data.

## How actually he SMOTE works?

- Select a minority class instance: It picks a random data point from the minority class.

- Find its neighbors: It identifies its k-nearest neighbors (other minority class points that are closest to it).

- Generate a new point: It creates a new, synthetic data point somewhere along the line segment between the original data point and one of its randomly selected neighbors.

This process is repeated until the minority class has a similar number of instances as the majority class, thus balancing the dataset.

In [15]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


X = df_encoded.drop(columns=['id', 'loan_status'])
y = df_encoded['loan_status']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Apply SMOTE to the training data only
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

model_smote = RandomForestClassifier(n_estimators=200, random_state=42)
model_smote.fit(X_train_smote, y_train_smote)

# Evaluate the model on the original, untouched test set
y_pred_smote = model_smote.predict(X_test)
print("Classification Report after applying SMOTE:")
print(classification_report(y_test, y_pred_smote))

Classification Report after applying SMOTE:
              precision    recall  f1-score   support

           0       0.96      0.98      0.97     10059
           1       0.84      0.73      0.78      1670

    accuracy                           0.94     11729
   macro avg       0.90      0.85      0.87     11729
weighted avg       0.94      0.94      0.94     11729



### Things are get better for class 1 :( :(

## Try logistic regression vs. decision tree

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix




print("--- Logistic Regression Model ---")
lr_model = LogisticRegression(random_state=42, solver='liblinear', class_weight='balanced')
lr_model.fit(X_train, y_train)

# Make predictions
lr_pred = lr_model.predict(X_test)

# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_test, lr_pred))
print("\nClassification Report:")
print(classification_report(y_test, lr_pred))


### 2. Train and Evaluate Decision Tree ###

print("\n--- Decision Tree Model ---")
dt_model = DecisionTreeClassifier(random_state=42, class_weight='balanced')
dt_model.fit(X_train, y_train)

# Make predictions
dt_pred = dt_model.predict(X_test)

# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_test, dt_pred))
print("\nClassification Report:")
print(classification_report(y_test, dt_pred))

--- Logistic Regression Model ---
Confusion Matrix:
[[8127 1960]
 [ 234 1408]]

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.81      0.88     10087
           1       0.42      0.86      0.56      1642

    accuracy                           0.81     11729
   macro avg       0.70      0.83      0.72     11729
weighted avg       0.89      0.81      0.84     11729


--- Decision Tree Model ---
Confusion Matrix:
[[9588  499]
 [ 465 1177]]

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.95      0.95     10087
           1       0.70      0.72      0.71      1642

    accuracy                           0.92     11729
   macro avg       0.83      0.83      0.83     11729
weighted avg       0.92      0.92      0.92     11729



## Final Observation: AS EXCPECTED F1-SCORE IS VERY BAD FOR LOGISITC REGRESSION,AS THE DATA ARE NON-LINEAR, AND FOR TREE IT IS VERY DECENT!